# Общие функции и константы для всех блоков модели

In [1]:
import sqlalchemy as sa
import warnings
from os import path, getcwd
import datetime as dt

warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt

In [23]:
class common:
    
    strYearDBPath=path.join('..','DB', 'year.sqlite3') # путь к рабочей базе данных фактических рядов SQLite 
    strExYearDBPath=path.join('..','DB', 'exog_year.sqlite3') # путь к рабочей базе данных экзогенных переменных
    strExParamDBPath=path.join('..','DB', 'exog_param.sqlite3') # путь к рабочей базе данных экзогенных параметров, задаваемых вручную
    strSvodDBPath=path.join('..','DB', 'svod.sqlite3') # путь к рабочей базе данных итоговых рядов SQLite 

    # константы базы данных SQLite3
    strDatas_table='datas' # название таблицы с данными в базе данных SQLite
    strHeaders_table='headers' # название таблицы с текстовыми и кодовыми именами рядов в базе данных SQLite
    
    def make_SELECT_YEAR_string(rowCodes, datas_table=strDatas_table, header_table=strHeaders_table):
        assert type(rowCodes) in (str, list, tuple), 'make_SELECT_YEAR_string from() COMMON module: error in param type'
        strSelect='''select {datas_table}.date, {datas_table}.value, {header_table}.code2 from {datas_table} join {header_table}
                      on {datas_table}.code={header_table}.code {where_condiiton}'''


        if type(rowCodes)==str:
            return strSelect.format(where_condiiton = 'where {header_table}.code2=="{code2}"'.format(header_table=header_table, code2=rowCodes),
                                   datas_table=datas_table, header_table=header_table)
        elif type(rowCodes) in (tuple, list):
            return strSelect.format(where_condiiton = 'where {header_table}.code2 in {codes2}'.format(header_table=header_table, 
                                                                                                      codes2=tuple(rowCodes)),
                                   datas_table=datas_table, header_table=header_table)
        return None

    def make_frame(pdf):
        k=pdf.set_index(['date', 'code2']).unstack().reset_index().set_index('date')
        k.columns=[c[1] for c in k.columns]
        return k

    def scale(pdf, list_fields=[], multiplier=1):
        pdf[list_fields] *= multiplier
        return pdf

    def add(pdf, list_fields=[], param=0):
        pdf[list_fields] += param
        return pdf
    
    def update_dt_s(datetime_value=dt.datetime.now()):
        return datetime_value.strftime('%Y-%m-%d %H:%M:%S %f')

    def update_dt_d(string_value=None):
        assert string_value is not None, 'на входе нужно строкое представление datetime в формате "%Y-%m-%d %H:%M:%S %f"'
        s=string_value.split(' ')
        l_dt=s[0].split('-')
        l_tm=s[1].split(':')
        i_msec=int(s[2])
        return dt.datetime(year=int(l_dt[0]), month=int(l_dt[1]), day=int(l_dt[2]), 
                           hour=int(l_tm[0]), minute=int(l_tm[1]), second=int(l_tm[2]), microsecond=int(i_msec))


## Ниже - тесты функций. Чтобы работало, нужно убрать магию

In [5]:
%%script false --no-raise-error

import pandas as pd

strDB_path=path.join('..', 'DB', 'bankrp.sqlite3')

strYearDBPath=path.join('..','DB', 'year.sqlite3') # путь к рабочей базе данных фактических рядов SQLite 
strExYearDBPath=path.join('..','DB', 'exog_year.sqlite3') # путь к рабочей базе данных экзогенных переменных SQLite
strExParamDBPath=path.join('..','DB', 'exog_param.sqlite3') # путь к рабочей базе данных экзогенных параметров SQLite
strSvodDBPath=path.join('..','DB', 'svod.sqlite3') # путь к рабочей базе данных итоговых рядов SQLite 


# ------------

conWork = sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=strYearDBPath)) # connection к рабочей базе данных

pdf=(pd.read_sql(common.make_SELECT_YEAR_string(['GDP_x', 'GDP_Iq','CPIAv','KeyRate','Unmpl_s']), con=conWork)
     .pipe(common.make_frame)
     .pipe(common.scale, list_fields=['GDP_x', 'GDP_Iq'], multiplier=100)
     .pipe(common.add, 'Unmpl_s', -100))
pdf

,CPIAv,GDP_Iq,GDP_x,KeyRate,Unmpl_s
date,,,,,
2004,1.108812,107.175949,1702.719086,0.135200,-99.922851
2005,1.127055,106.376187,2160.976549,0.129800,-99.928468
2006,1.096830,108.153432,2691.720138,0.088942,-99.929378
2007,1.089881,108.535080,3324.751323,0.075126,-99.939706
2008,1.141048,105.247954,4127.684919,0.081291,-99.937588
2009,1.116977,92.179115,3880.721857,0.113100,-99.917478
2010,1.068508,104.503726,4630.854119,0.080300,-99.926555
2011,1.084634,104.264177,6011.400084,0.081200,-99.935044
2012,1.050667,104.024086,6810.344963,0.080700,-99.945413


219273946611658146308320240553143687149